In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from utils import *
import itertools

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path_to_train = 'plant-seedlings-classification/train'
path_to_test = 'plant-seedlings-classification/test'

In [ ]:
classes = []
for subfolder in os.listdir(path_to_train):
    classes.append((subfolder, len(os.listdir(os.path.join(path_to_train, subfolder)))))

df_class = pd.DataFrame(classes, columns=['class', 'count'])
df_class['percentage'] = df_class['count'] * 100 / df_class['count'].sum()
df_class['percentage'] = df_class['percentage'].round(2)
df_class

Classes are not balanced.

In [ ]:
def plot_image_grid(folder_path, images_per_class = 4):
    # Create a figure
    fig = plt.figure(figsize=(images_per_class*2, images_per_class*8))

    # List subfolders
    classes = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    num_classes = len(classes)

    for i, class_name in enumerate(classes):
        class_folder = os.path.join(folder_path, class_name)

        # Get image files
        images = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
        selected_images = images[:images_per_class]

        for j, image in enumerate(selected_images):
            # Read and plot each image
            img = imread(os.path.join(class_folder, image))
            ax = fig.add_subplot(num_classes, images_per_class, i * images_per_class + j + 1)
            ax.imshow(img)
            ax.axis('off')
            if j == 0:
                ax.set_title(class_name)

    plt.tight_layout()
    plt.show()

# plot_image_grid(path_to_train)

In [ ]:
mlp_configurations = [
    ([512, 256], 'mlp_n'),
    ([1024, 512, 256], 'mlp_s'),
    ([2048, 1024, 512, 256], 'mlp_m'),
    ([4096, 2048, 1024, 512, 256], 'mlp_l'),
    ([8192, 4096, 2048, 1024, 512, 256], 'mlp_xl'),
]

cnn_configurations = [
    ([(16, 3, 1, 1), (32, 3, 1, 1)], [64], 'cnn_small'),
    ([(32, 3, 1, 1), (64, 3, 1, 1), (128, 3, 1, 1)], [128, 64], 'cnn_medium'),
    ([(64, 6, 1, 1), (128, 3, 1, 1), (256, 3, 1, 1), (512, 3, 1, 1)], [256, 128, 64], 'cnn_large'),
    ([(128, 6, 1, 1), (256, 6, 1, 1), (512, 3, 1, 1), (1024, 3, 1, 1), (2048, 3, 1, 1)], [512, 256, 128, 64], 'cnn_extra_large')
]

pretrained_models = [
    'efficientnet_b0',
    'efficientnet_b1',
    'efficientnet_b2',
    'efficientnet_b3',
    'efficientnet_b4',
    'resnet18',
    'resnet34',
    'resnet50',
    'resnet101',
    'mobilenet_v2',
    'googlenet',
]

# Train

In [ ]:
image_size = 128
for config in mlp_configurations + cnn_configurations:
    model_name = config[-1]
    if 'mlp' in model_name:
        model = create_mlp_model(config, image_size)
    else:
        model = create_cnn_model(config, image_size)
    
    model = model.to(device)

    train(
        path_to_train,
        model,
        model_name,
        lr0 = 0.0001,
        lrf = 0.00001,
        weight_decay = 0.00005,
        num_epochs = 100,
        patience = 10,
        batch_size = 16,
        image_size = image_size,
        val_size = 0.15,
        device = device,
    )

    temp_dataset = datasets.ImageFolder(path_to_train)
    class2id = temp_dataset.class_to_idx

    # load the best model
    model.load_state_dict(torch.load(f'{model_name}/{model_name}_best_model.pt'))

    test_submit(model, model_name, image_size, path_to_test, class2id, device)

    del model

In [ ]:
image_size = 256

for model_name in pretrained_models:
    if 'efficientnet' in model_name:
        model = create_efficientnet_model(model_name)
    elif 'resnet' in model_name:
        model = create_resnet_model(model_name)
    elif 'mobilenet' in model_name:
        model = create_mobile_net_model(model_name)
    elif 'googlenet' in model_name:
        model = create_googlenet_model(model_name)
    else:
        raise Exception('Unknown model name')
    
    model = model.to(device)

    train(
        path_to_train,
        model,
        model_name,
        lr0 = 0.0001,
        lrf = 0.00001,
        weight_decay = 0.00005,
        num_epochs = 100,
        patience = 10,
        batch_size = 64,
        image_size = image_size,
        val_size = 0.15,
        device = device,
    )

    temp_dataset = datasets.ImageFolder(path_to_train)
    class2id = temp_dataset.class_to_idx

    # load the best model
    model.load_state_dict(torch.load(f'{model_name}/{model_name}_best_model.pt'))

    test_submit(model, model_name, image_size, path_to_test, class2id, device)

    del model

In [ ]:
temp_dataset = datasets.ImageFolder(path_to_train)
class2id = temp_dataset.class_to_idx

predict_voting(['efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2'], 256, path_to_test, class2id, device)